In [4]:
import os
import sys
import torch
import numpy as np
from dataset import LABEL_MAP
from sklearn.metrics import accuracy_score, confusion_matrix

from dataset import SentimentAnalysisDataset

import torch

from model import TransformerModel, SentimentGRUWithGlove
from dataset import SentimentAnalysisDataset

def run_test(model, dataset, loss_func, device):

    test_loss = 0.0
    y_true = []
    y_predict = []

    model.eval()
    with torch.no_grad():
        for tokens, label in dataset:
            
            tokens = tokens.to(device).float()
            label = label.to(device)

            y_true.append(int(label.argmax()))

            if tokens.squeeze().dim() == 0 or len(tokens.squeeze()) == 0:
                # Predict neutral if no token after processing 
                # e.g., only stopwords in the original text
                y_predict.append(int(LABEL_MAP['neutral']))
                continue

            # Forward pass
            output = model(tokens)
            y_predict.append(int(torch.softmax(output, dim=0).argmax()))
            L = loss_func(output.view(1,-1), label.view(1,-1))
            test_loss += L.item()

    y_true = np.array(y_true)
    y_predict = np.array(y_predict)

    return accuracy_score(y_true, y_predict), confusion_matrix(y_true, y_predict), test_loss

====== TEST =======
device: cpu
-> Loading vocabulary
-> Loading datasets
-> Initalizing model


In [10]:
data_path = 'data/testEmotions.csv'
model_type = 'transformer'
model_path = ''

test_dataset = SentimentAnalysisDataset(data_path)
test_size = len(test_dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if model_type == 'gru':
    model = SentimentGRUWithGlove(100)
else:
    model = TransformerModel(100)

model.load_state_dict(torch.load(model_path))
model.to(device)

loss = torch.nn.CrossEntropyLoss()

In [11]:
acc, conf_mat = run_test(model, test_dataset, loss, device)
print('accuracy:', acc)
print('confusion matrix:', conf_mat)

((4837,), (4837,))